In [ ]:
!virtualenv test_env1
!.\test_env1\Scripts\activate
from bs4 import BeautifulSoup # for scrapping  information from the web
import requests # for making the requests on web
import os # for implementing genai
import google.generativeai as genai #for gemini
import gradio as gr #for gradio
from dotenv import load_dotenv
load_dotenv()
def webscrapping(url):
    #url = input("Enter URL : ")
    #scraping links from the website
    page_to_scrape = requests.get(url) #This loads the URL into page_to_scrape

    file =  open("website_in_textformat.txt","w",encoding="utf-8")
    file.write(page_to_scrape.text)
    file.close()
    soup = BeautifulSoup(page_to_scrape.text,"html.parser")
    # selecting the data from website to scrape
    headings = soup.find("div",attrs={"class": "sp-ttl-wrp"}).find("h1", attrs = {"class":"sp-ttl"}).get_text()
    # printing the heading 
    #print("Heading : ",headings)
    #printing the title
    para = soup.find("div", attrs = {"class": "sp-cn ins_storybody"}).find_all("p")
    para_var = ""
    for i in para:
        para_var = para_var +i.getText()
    #print(para_var)
    genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
    #  setting technical parameters  into  a variable, based on which the genai will generate my response 
    generation_config = {
      "temperature": 0.9,
      "top_p": 0.95,
      "top_k": 64,
      "max_output_tokens": 8192,
      "response_mime_type": "text/plain"
    }

    # feeding the generation_config and setting the model_name to be used for gen ai
    model = genai.GenerativeModel(model_name="gemini-1.5-pro", generation_config=generation_config)
    #sometimes when the text is too long , it can throw some error!
    response = model.generate_content([f"Summarize the following paragraph for me : {para_var}"])
    #print(response.text)
    return response.text



interface_web = gr.Interface(fn=webscrapping, inputs=gr.Textbox(label='Enter the NDTV url'), outputs='text')
interface_web.launch(share='True')